# Align Dbit-seq transcriptome data and images using Squidpy tools

In [1]:
## The following code ensures that all functions and init files are reloaded before executions.
%load_ext autoreload
%autoreload 2

In [2]:
%gui qt

In [3]:
## Import the custom library
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import dbitx as db

In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
import scanpy as sc
import json
import squidpy as sq
import napari

### Specify paths

In [71]:
# input parameters
well_id = 0
alignment_id = 1
image_names = ['bf', 'align', 'phalloidin', 'dapi']

# image path
#image_path = r"N:\01 HPC\03 Team Meier\10_Resources\08_Johannes Wirth\Nextcloud\DbitX\data\37_30\images"
image_path = "/Users/Johannes/Nextcloud/DbitX/data/37_30/images/"

# transcriptome path
matrix_path = r"N:\01 HPC\03 Team Meier\10_Resources\08_Johannes Wirth\Nextcloud\DbitX\data\37_30\transcriptome\wells\A1\DGE_matrix_with_introns_min100.txt.gz"

# output directory
output_name = "37_28_adata_raw_with_images.h5ad"

In [66]:
# get image names
well_names = os.listdir(image_path)
well_name = well_names[well_id]
image_names = os.listdir(os.path.join(image_path,well_name))

In [67]:
# load images
images = [cv2.imread(os.path.join(image_path, well_name, name), -1) for name in image_names]

In [68]:
alignment_image = images[alignment_id]

### Select corner spots in alignment image using napari viewer

In [74]:
viewer = napari.view_image(alignment_image, title="Select corner spots in alignment image of well " + well_name)

#### Extract pivot spot and calibration points from viewer

In [70]:
# fetch vertices (center points of )
corner_spots_center = viewer.layers["Points"].data.astype(int)

In [26]:
# values from 37_28
#corner_spots_center = np.array([[ 486,  990],[ 891, 8662], [8567, 8324],[8164,  646]])

### Create adata with images and metadata

In [73]:
savepath = os.path.join(well_path, output_name)

adata = db.dbitseq_to_squidpy(matrix_path=matrix_path, images=images, labels=image_names, vertices=corner_spots_center, 
                              resolution=50, n_channels=38, frame=100, savepath=savepath)

Read transcriptome matrix...
Align and create image metadata...
     Align images...
     Create metadata...
     Summarized aligned images and metadata...
Compute coordinates...
Adata object generated.
Saving image...
Adata object saved into N:\01 HPC\03 Team Meier\10_Resources\08_Johannes Wirth\Nextcloud\DbitX\data\37_30\images\A1\37_28_adata_raw_with_images.h5ad
Finished


## Write total data to file

In [43]:
adata.write(os.path.join(well_path, output_name))